In [1]:
import tensorflow as tf
print(tf.__version__)

1.8.0


In [2]:
dataset = tf.data.Dataset.range(10)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(10):
        value = sess.run(next_element)
        print(f"{value} ", end=" ")

0  1  2  3  4  5  6  7  8  9  

In [3]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.repeat(2)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(10):
        value = sess.run(next_element)
        print(f"{value} ", end=" ")

0  1  2  3  4  0  1  2  3  4  

In [4]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)    # Take a placeholder to create a dataset
iterator = dataset.make_initializable_iterator()      # Create an initializable iterator
next_element = iterator.get_next()

with tf.Session() as sess:
    # Initialize an iterator over a dataset with 10 elements using placeholder.
    sess.run(iterator.initializer, feed_dict={max_value: 10}) 

    for _ in range(10):
        value = sess.run(next_element)
        print(f"{value} ", end=" ")

0  1  2  3  4  5  6  7  8  9  

In [20]:
training_dataset = tf.data.Dataset.range(10)
validation_dataset = tf.data.Dataset.range(5)

# Build an iterator that can take different datasets with the same type and shape
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# Get 2 init op for 2 different dataset
training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
    for _ in range(2):
        sess.run(training_init_op)
        for _ in range(10):
            value = sess.run(next_element)
            print(f"{value} ", end=" ")

        sess.run(validation_init_op)
        for _ in range(5):
            value = sess.run(next_element)
            print(f"{value} ", end=" ")
            
        print('\n')

0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  

0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  



In [21]:
# Create 2 dataset witht the same datatype and shape
training_dataset = tf.data.Dataset.range(30)
validation_dataset = tf.data.Dataset.range(5)

# Create a feedable iterator that use a placeholder to switch between dataset
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# Create 2 iterators
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

with tf.Session() as sess:
    # Return handles that can be feed as the iterator in sess.run
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())

    for _ in range(3):
        for _ in range(10):
            value = sess.run(next_element, feed_dict={handle: training_handle})
            print(f"{value} ", end=" ")

        sess.run(validation_iterator.initializer)
        for _ in range(5):
            value = sess.run(next_element, feed_dict={handle: validation_handle})
            print(f"{value} ", end=" ")
        
        print('\n')

0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  

10  11  12  13  14  15  16  17  18  19  0  1  2  3  4  

20  21  22  23  24  25  26  27  28  29  0  1  2  3  4  



In [7]:
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))

iterator = dataset.make_initializable_iterator()      # Create an initializable iterator
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)

    for i in range(4):
        value = sess.run(next_element)
        print(f"{value} ")        # Print out an array with 10 random numbers

[0.09447801 0.5290395  0.46006417 0.03894734 0.470865   0.28260875
 0.5192795  0.59933877 0.7981429  0.3622675 ] 
[0.36460412 0.46807122 0.3752532  0.70037234 0.19405639 0.8199223
 0.03183115 0.19940841 0.48549366 0.4922037 ] 
[0.66499364 0.06752551 0.06007934 0.70466506 0.00995421 0.72531235
 0.64778054 0.63643646 0.18187034 0.01640058] 
[0.14747453 0.7501242  0.03595364 0.8796506  0.03570914 0.98773813
 0.16869736 0.9395461  0.85256135 0.7036263 ] 


In [8]:
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),                                   # (tf.float32, tf.int32)
     tf.random_uniform([4, 100], maxval=100, dtype=tf.int32))) # ((), (100,))

iterator = dataset.make_initializable_iterator()      # Create an initializable iterator
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)

    for i in range(4):
        value = sess.run(next_element)
        print(f"{value} ")        # Print out an array with 10 random numbers

(0.5518974, array([83, 41, 96, 56, 26, 10,  7, 88, 21, 91, 11, 87, 10, 39, 63, 85, 26,
       56, 63, 19, 93, 15, 28, 52, 65, 59,  3, 40, 69, 15, 50, 28, 73, 97,
       87, 81, 86,  7,  8, 84, 61, 74, 77, 71, 20, 57, 98, 15, 71, 85, 49,
       93,  6, 94, 13, 47, 80, 78, 24, 52, 52, 20, 54, 88, 68, 38, 80, 80,
        0, 66, 86,  8, 99, 57,  8, 41, 83, 66, 66, 59, 83, 90, 72, 29, 62,
       62, 20,  0,  2, 51, 71, 92, 97, 11, 72,  6, 35, 73, 30, 91],
      dtype=int32)) 
(0.20816946, array([43, 80, 61, 74, 31, 34, 46, 74, 98, 38, 71,  7, 69, 45, 84, 38, 28,
       35,  0, 56, 73, 49, 13,  5, 66, 57, 36, 64, 61, 24, 62,  5, 99, 41,
        1, 23, 89,  3, 16, 64, 65, 11, 50, 74, 38,  9, 21, 50, 92, 42, 30,
        4, 72, 44, 39, 42, 85, 60, 60, 57, 88,  3, 70, 47, 20, 61, 90, 66,
       88, 76, 49,  3, 64, 44, 94, 82, 24,  5, 38, 21, 17, 61, 17, 80, 69,
       65, 72, 78, 54, 73, 10, 80, 81, 88, 25, 82,  1, 81,  7, 83],
      dtype=int32)) 
(0.98128736, array([30, 16, 81, 77, 79, 69, 30,

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),                                   # (tf.float32, tf.int32)
     tf.random_uniform([4, 100], maxval=100, dtype=tf.int32))) # ((), (100,))
print(dataset.output_types)
print(dataset.output_shapes)

(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))


In [10]:
dataset = tf.data.Dataset.from_tensor_slices(
    {"a": tf.random_uniform([4]),
     "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes) # ==> "{'a': (), 'b': (100,)})

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


In [24]:
filenames = ["wine_data.csv"]

dataset = tf.data.Dataset.from_tensor_slices(filenames)
dataset = dataset.flat_map(
    lambda filename: (
        tf.data.TextLineDataset(filename)
        .skip(1)
        ))

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(2):
        value = sess.run(next_element)
        print(f"{value}", end="\n")

b'0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022\xe2\x80\x932030.",Martha\'s Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz'
b'1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selecci\xc3\xb3n Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodr\xc3\xadguez'


In [26]:
filename = "wine_data.csv"

dataset = (
        tf.data.TextLineDataset(filename)
        .skip(1)
        )

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(2):
        value = sess.run(next_element)
        print(f"{value}", end="\n")

b'0,US,"This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022\xe2\x80\x932030.",Martha\'s Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz'
b'1,Spain,"Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.",Carodorum Selecci\xc3\xb3n Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodr\xc3\xadguez'


In [30]:
filename = "wine_data.csv"

def decode_csv(line):
    record_defaults = [[0], [""], [""], [""], [""], [0], [0.], [""], [""], [""], [""], [""]]
    parsed_line = tf.decode_csv(line, record_defaults)
    #label = parsed_line[-1:] # Last element is the label
    #del parsed_line[-1] # Delete last element
    #features = parsed_line # Everything (but last element) are the features
    #d = dict(zip(feature_names, features)), label
    return parsed_line

dataset = (
        tf.data.TextLineDataset(filename)
        .skip(1)
        .map(decode_csv)
        )

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for _ in range(2):
        value = sess.run(next_element)
        print(f"{value}", end="\n")

InvalidArgumentError: Expect 12 fields but have 11 in record 0
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_INT32, DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_INT32, DT_FLOAT, DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_STRING], field_delim=",", na_value="", use_quote_delim=true](arg0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_1, DecodeCSV/record_defaults_2, DecodeCSV/record_defaults_3, DecodeCSV/record_defaults_4, DecodeCSV/record_defaults_5, DecodeCSV/record_defaults_6, DecodeCSV/record_defaults_7, DecodeCSV/record_defaults_8, DecodeCSV/record_defaults_9, DecodeCSV/record_defaults_10, DecodeCSV/record_defaults_11)]]
	 [[Node: IteratorGetNext_17 = IteratorGetNext[output_shapes=[[], [], [], [], [], [], [], [], [], [], [], []], output_types=[DT_INT32, DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_INT32, DT_FLOAT, DT_STRING, DT_STRING, DT_STRING, DT_STRING, DT_STRING], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_12)]]

Caused by op 'DecodeC